
<h1 align=center><font size = 5>Project Regression Models with Keras</font></h1>




<h3>Objective for this Notebook: Project Course DL with Keras - Regression Model<h3>    

<h3> Module A </h3>

<h5> 1. Download and Clean dataset </h5>
<h5> 2. Build a Neural Network </h5>
<h5> 3. Train and Test the Network. </h5>     

<h3> Module B. Repeat Normalizing data </h3> 

<h3> Module C. With 100 epochs </h3> 

<h3> Module D. Three Hidden Layers with 10 nodes </h3>



## Module A


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np


<strong>As required for the project, the dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


<em> So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. 

### <em>Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [28]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [7]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to _n_cols_ since we will need this number when building our network.


In [8]:
n_cols = predictors.shape[1] # number of predictors

<a id="item1"></a>


<a id='item32'></a>


## Import Keras


Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, I need to install first Tensorflow.


#### Let's go ahead and import the Keras library


In [9]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [10]:
import tensorflow as tf
import keras

As you can see, the TensorFlow backend was used to install the Keras library.


Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [11]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [22]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

### The above function create a model that has ONE hidden layer, of 10 nodes.


<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [23]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model for 50 epochs.


In [14]:
# fit the model
model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=2)

Epoch 1/50
23/23 - 0s - loss: 399682.5000 - val_loss: 285509.4375
Epoch 2/50
23/23 - 0s - loss: 233331.2812 - val_loss: 161474.0625
Epoch 3/50
23/23 - 0s - loss: 128559.7578 - val_loss: 84821.3359
Epoch 4/50
23/23 - 0s - loss: 63574.1875 - val_loss: 38827.5078
Epoch 5/50
23/23 - 0s - loss: 27242.2129 - val_loss: 16257.1699
Epoch 6/50
23/23 - 0s - loss: 11050.6670 - val_loss: 7763.3140
Epoch 7/50
23/23 - 0s - loss: 5834.4805 - val_loss: 5445.6201
Epoch 8/50
23/23 - 0s - loss: 4526.2290 - val_loss: 4895.9497
Epoch 9/50
23/23 - 0s - loss: 4159.0747 - val_loss: 4629.0298
Epoch 10/50
23/23 - 0s - loss: 3913.6099 - val_loss: 4392.4023
Epoch 11/50
23/23 - 0s - loss: 3676.5051 - val_loss: 4154.2104
Epoch 12/50
23/23 - 0s - loss: 3450.0461 - val_loss: 3940.2014
Epoch 13/50
23/23 - 0s - loss: 3240.9053 - val_loss: 3725.5679
Epoch 14/50
23/23 - 0s - loss: 3036.3333 - val_loss: 3525.0720
Epoch 15/50
23/23 - 0s - loss: 2848.3030 - val_loss: 3331.7129
Epoch 16/50
23/23 - 0s - loss: 2667.4392 - val_l


"loss" is the mean squared error of Train set, and "val_loss" is the mean squared error of Test Set for validation.

We see that for Training Set the mean squared error is normally decreasing each epoch and it finish in 345.9, but in Test Set this reduction is not so clear or constant and mean squared error is higher than for Training Set, finishing in 469.7.

(These values will change if the fit process is executed again)

To repeat 50 times this process for fit, we use the following loop, and we'll observe how "loss" and "val-loss" evolve.


In [15]:
for i in np.arange(50):
    model.fit(predictors, target, validation_split=0.3, epochs=50, verbose=2)
    print(i)
    i=i+1
    

Epoch 1/50
23/23 - 0s - loss: 329.7490 - val_loss: 441.5069
Epoch 2/50
23/23 - 0s - loss: 314.0118 - val_loss: 421.7927
Epoch 3/50
23/23 - 0s - loss: 301.1889 - val_loss: 399.2872
Epoch 4/50
23/23 - 0s - loss: 287.4913 - val_loss: 377.3363
Epoch 5/50
23/23 - 0s - loss: 275.5815 - val_loss: 358.4194
Epoch 6/50
23/23 - 0s - loss: 264.2408 - val_loss: 340.2987
Epoch 7/50
23/23 - 0s - loss: 254.2651 - val_loss: 322.5424
Epoch 8/50
23/23 - 0s - loss: 244.5639 - val_loss: 308.8388
Epoch 9/50
23/23 - 0s - loss: 235.9784 - val_loss: 292.3566
Epoch 10/50
23/23 - 0s - loss: 227.1628 - val_loss: 277.9875
Epoch 11/50
23/23 - 0s - loss: 219.2915 - val_loss: 263.7850
Epoch 12/50
23/23 - 0s - loss: 212.2295 - val_loss: 252.1653
Epoch 13/50
23/23 - 0s - loss: 205.3715 - val_loss: 242.0554
Epoch 14/50
23/23 - 0s - loss: 198.9566 - val_loss: 228.8040
Epoch 15/50
23/23 - 0s - loss: 193.7198 - val_loss: 220.1222
Epoch 16/50
23/23 - 0s - loss: 188.7636 - val_loss: 209.6974
Epoch 17/50
23/23 - 0s - loss: 18

Epoch 38/50
23/23 - 0s - loss: 125.4369 - val_loss: 78.7345
Epoch 39/50
23/23 - 0s - loss: 125.4865 - val_loss: 78.0645
Epoch 40/50
23/23 - 0s - loss: 125.3335 - val_loss: 77.7146
Epoch 41/50
23/23 - 0s - loss: 126.5184 - val_loss: 77.4438
Epoch 42/50
23/23 - 0s - loss: 126.0349 - val_loss: 79.8210
Epoch 43/50
23/23 - 0s - loss: 126.7167 - val_loss: 78.9873
Epoch 44/50
23/23 - 0s - loss: 124.8581 - val_loss: 79.3375
Epoch 45/50
23/23 - 0s - loss: 125.5531 - val_loss: 84.2992
Epoch 46/50
23/23 - 0s - loss: 125.1619 - val_loss: 77.0740
Epoch 47/50
23/23 - 0s - loss: 125.4888 - val_loss: 77.4397
Epoch 48/50
23/23 - 0s - loss: 124.7880 - val_loss: 77.4411
Epoch 49/50
23/23 - 0s - loss: 126.7434 - val_loss: 77.4861
Epoch 50/50
23/23 - 0s - loss: 126.7932 - val_loss: 76.8086
2
Epoch 1/50
23/23 - 0s - loss: 125.4917 - val_loss: 77.0759
Epoch 2/50
23/23 - 0s - loss: 125.3570 - val_loss: 77.5277
Epoch 3/50
23/23 - 0s - loss: 125.5700 - val_loss: 77.2635
Epoch 4/50
23/23 - 0s - loss: 127.4847 - 

Epoch 26/50
23/23 - 0s - loss: 85.7445 - val_loss: 62.7853
Epoch 27/50
23/23 - 0s - loss: 89.4806 - val_loss: 60.8657
Epoch 28/50
23/23 - 0s - loss: 85.8672 - val_loss: 58.8516
Epoch 29/50
23/23 - 0s - loss: 84.0835 - val_loss: 59.9377
Epoch 30/50
23/23 - 0s - loss: 85.2583 - val_loss: 60.2483
Epoch 31/50
23/23 - 0s - loss: 87.0591 - val_loss: 61.4105
Epoch 32/50
23/23 - 0s - loss: 83.7087 - val_loss: 57.9712
Epoch 33/50
23/23 - 0s - loss: 84.2936 - val_loss: 61.2564
Epoch 34/50
23/23 - 0s - loss: 82.9835 - val_loss: 60.7545
Epoch 35/50
23/23 - 0s - loss: 82.1961 - val_loss: 58.2310
Epoch 36/50
23/23 - 0s - loss: 82.3686 - val_loss: 65.0473
Epoch 37/50
23/23 - 0s - loss: 85.4466 - val_loss: 58.4907
Epoch 38/50
23/23 - 0s - loss: 86.2606 - val_loss: 63.8688
Epoch 39/50
23/23 - 0s - loss: 83.4062 - val_loss: 60.5874
Epoch 40/50
23/23 - 0s - loss: 80.3575 - val_loss: 59.3918
Epoch 41/50
23/23 - 0s - loss: 81.0333 - val_loss: 63.1842
Epoch 42/50
23/23 - 0s - loss: 82.4780 - val_loss: 60.96

Epoch 16/50
23/23 - 0s - loss: 76.2564 - val_loss: 58.6194
Epoch 17/50
23/23 - 0s - loss: 73.7068 - val_loss: 57.5999
Epoch 18/50
23/23 - 0s - loss: 76.8895 - val_loss: 86.0727
Epoch 19/50
23/23 - 0s - loss: 78.3888 - val_loss: 61.5704
Epoch 20/50
23/23 - 0s - loss: 78.0144 - val_loss: 57.7825
Epoch 21/50
23/23 - 0s - loss: 74.5828 - val_loss: 60.2438
Epoch 22/50
23/23 - 0s - loss: 76.0918 - val_loss: 58.9693
Epoch 23/50
23/23 - 0s - loss: 75.6213 - val_loss: 63.4070
Epoch 24/50
23/23 - 0s - loss: 75.4773 - val_loss: 58.4849
Epoch 25/50
23/23 - 0s - loss: 76.5668 - val_loss: 58.5815
Epoch 26/50
23/23 - 0s - loss: 75.6337 - val_loss: 62.3937
Epoch 27/50
23/23 - 0s - loss: 75.1222 - val_loss: 57.9552
Epoch 28/50
23/23 - 0s - loss: 74.5246 - val_loss: 62.5285
Epoch 29/50
23/23 - 0s - loss: 75.7046 - val_loss: 63.0433
Epoch 30/50
23/23 - 0s - loss: 75.2001 - val_loss: 57.8432
Epoch 31/50
23/23 - 0s - loss: 73.8556 - val_loss: 57.0298
Epoch 32/50
23/23 - 0s - loss: 76.2080 - val_loss: 61.95

Epoch 6/50
23/23 - 0s - loss: 57.3856 - val_loss: 60.0179
Epoch 7/50
23/23 - 0s - loss: 56.6848 - val_loss: 76.8013
Epoch 8/50
23/23 - 0s - loss: 61.1288 - val_loss: 83.2991
Epoch 9/50
23/23 - 0s - loss: 57.4356 - val_loss: 75.0428
Epoch 10/50
23/23 - 0s - loss: 56.5695 - val_loss: 78.2061
Epoch 11/50
23/23 - 0s - loss: 58.2630 - val_loss: 64.6724
Epoch 12/50
23/23 - 0s - loss: 55.5919 - val_loss: 58.7925
Epoch 13/50
23/23 - 0s - loss: 56.8935 - val_loss: 62.5645
Epoch 14/50
23/23 - 0s - loss: 58.0396 - val_loss: 78.3272
Epoch 15/50
23/23 - 0s - loss: 57.1379 - val_loss: 56.0941
Epoch 16/50
23/23 - 0s - loss: 56.9497 - val_loss: 57.5776
Epoch 17/50
23/23 - 0s - loss: 55.8511 - val_loss: 64.3541
Epoch 18/50
23/23 - 0s - loss: 59.0756 - val_loss: 117.3336
Epoch 19/50
23/23 - 0s - loss: 61.6844 - val_loss: 87.7160
Epoch 20/50
23/23 - 0s - loss: 56.4045 - val_loss: 57.0219
Epoch 21/50
23/23 - 0s - loss: 54.9773 - val_loss: 60.3223
Epoch 22/50
23/23 - 0s - loss: 55.9927 - val_loss: 65.8184


...

23/23 - 0s - loss: 45.1020 - val_loss: 114.6010
Epoch 43/50
23/23 - 0s - loss: 45.4652 - val_loss: 107.9849
Epoch 44/50
23/23 - 0s - loss: 46.1387 - val_loss: 123.9744
Epoch 45/50
23/23 - 0s - loss: 46.5203 - val_loss: 90.7987
Epoch 46/50
23/23 - 0s - loss: 44.2639 - val_loss: 104.9235
Epoch 47/50
23/23 - 0s - loss: 45.5820 - val_loss: 121.0718
Epoch 48/50
23/23 - 0s - loss: 45.5483 - val_loss: 88.5693
Epoch 49/50
23/23 - 0s - loss: 46.0374 - val_loss: 89.5070
Epoch 50/50
23/23 - 0s - loss: 44.3899 - val_loss: 99.7155
38
Epoch 1/50
23/23 - 0s - loss: 46.5747 - val_loss: 91.2623
Epoch 2/50
23/23 - 0s - loss: 45.4368 - val_loss: 99.1805
Epoch 3/50
23/23 - 0s - loss: 45.1820 - val_loss: 94.4460
Epoch 4/50
23/23 - 0s - loss: 47.0593 - val_loss: 99.5359
Epoch 5/50
23/23 - 0s - loss: 47.8080 - val_loss: 97.3216
Epoch 6/50
23/23 - 0s - loss: 47.6126 - val_loss: 89.7071
Epoch 7/50
23/23 - 0s - loss: 46.9159 - val_loss: 126.7001
Epoch 8/50
23/23 - 0s - loss: 48.3377 - val_loss: 128.4792
Epoch 9

Epoch 31/50
23/23 - 0s - loss: 46.0810 - val_loss: 101.6398
Epoch 32/50
23/23 - 0s - loss: 44.9295 - val_loss: 96.0048
Epoch 33/50
23/23 - 0s - loss: 45.3794 - val_loss: 103.2143
Epoch 34/50
23/23 - 0s - loss: 45.4432 - val_loss: 112.0740
Epoch 35/50
23/23 - 0s - loss: 44.8080 - val_loss: 101.9798
Epoch 36/50
23/23 - 0s - loss: 45.2134 - val_loss: 106.1823
Epoch 37/50
23/23 - 0s - loss: 44.3183 - val_loss: 112.8737
Epoch 38/50
23/23 - 0s - loss: 47.5774 - val_loss: 125.7201
Epoch 39/50
23/23 - 0s - loss: 44.9397 - val_loss: 109.6145
Epoch 40/50
23/23 - 0s - loss: 44.4870 - val_loss: 97.6364
Epoch 41/50
23/23 - 0s - loss: 46.0999 - val_loss: 77.1187
Epoch 42/50
23/23 - 0s - loss: 47.5512 - val_loss: 79.0250
Epoch 43/50
23/23 - 0s - loss: 47.5837 - val_loss: 79.2560
Epoch 44/50
23/23 - 0s - loss: 44.8113 - val_loss: 87.4361
Epoch 45/50
23/23 - 0s - loss: 44.5270 - val_loss: 90.5357
Epoch 46/50
23/23 - 0s - loss: 44.4933 - val_loss: 107.1430
Epoch 47/50
23/23 - 0s - loss: 44.2987 - val_lo

Epoch 20/50
23/23 - 0s - loss: 44.5788 - val_loss: 109.6965
Epoch 21/50
23/23 - 0s - loss: 43.9829 - val_loss: 92.7990
Epoch 22/50
23/23 - 0s - loss: 43.8085 - val_loss: 92.1068
Epoch 23/50
23/23 - 0s - loss: 45.8474 - val_loss: 127.5470
Epoch 24/50
23/23 - 0s - loss: 45.6709 - val_loss: 111.3202
Epoch 25/50
23/23 - 0s - loss: 43.8002 - val_loss: 95.6702
Epoch 26/50
23/23 - 0s - loss: 44.4988 - val_loss: 80.1462
Epoch 27/50
23/23 - 0s - loss: 53.2865 - val_loss: 73.7684
Epoch 28/50
23/23 - 0s - loss: 46.6910 - val_loss: 97.7098
Epoch 29/50
23/23 - 0s - loss: 45.1064 - val_loss: 93.0264
Epoch 30/50
23/23 - 0s - loss: 45.8109 - val_loss: 77.0416
Epoch 31/50
23/23 - 0s - loss: 48.3898 - val_loss: 83.1419
Epoch 32/50
23/23 - 0s - loss: 46.8582 - val_loss: 111.3472
Epoch 33/50
23/23 - 0s - loss: 45.2275 - val_loss: 128.6268
Epoch 34/50
23/23 - 0s - loss: 46.7424 - val_loss: 101.2546
Epoch 35/50
23/23 - 0s - loss: 43.8271 - val_loss: 102.2435
Epoch 36/50
23/23 - 0s - loss: 45.9410 - val_loss

Epoch 8/50
23/23 - 0s - loss: 46.4434 - val_loss: 118.5103
Epoch 9/50
23/23 - 0s - loss: 44.2383 - val_loss: 97.9676
Epoch 10/50
23/23 - 0s - loss: 43.4869 - val_loss: 99.5323
Epoch 11/50
23/23 - 0s - loss: 43.8640 - val_loss: 112.6028
Epoch 12/50
23/23 - 0s - loss: 44.4459 - val_loss: 110.8246
Epoch 13/50
23/23 - 0s - loss: 44.0374 - val_loss: 96.8186
Epoch 14/50
23/23 - 0s - loss: 44.9579 - val_loss: 97.5462
Epoch 15/50
23/23 - 0s - loss: 45.1088 - val_loss: 115.6506
Epoch 16/50
23/23 - 0s - loss: 45.9881 - val_loss: 81.1419
Epoch 17/50
23/23 - 0s - loss: 44.6528 - val_loss: 110.1764
Epoch 18/50
23/23 - 0s - loss: 44.4793 - val_loss: 75.7554
Epoch 19/50
23/23 - 0s - loss: 45.9227 - val_loss: 96.0220
Epoch 20/50
23/23 - 0s - loss: 44.3486 - val_loss: 86.6170
Epoch 21/50
23/23 - 0s - loss: 52.3028 - val_loss: 74.4227
Epoch 22/50
23/23 - 0s - loss: 46.6762 - val_loss: 88.2405
Epoch 23/50
23/23 - 0s - loss: 44.5633 - val_loss: 91.9499
Epoch 24/50
23/23 - 0s - loss: 45.1273 - val_loss: 11

Epoch 46/50
23/23 - 0s - loss: 44.5510 - val_loss: 97.3544
Epoch 47/50
23/23 - 0s - loss: 45.5776 - val_loss: 125.2650
Epoch 48/50
23/23 - 0s - loss: 45.8825 - val_loss: 134.3704
Epoch 49/50
23/23 - 0s - loss: 45.3242 - val_loss: 102.0771
Epoch 50/50
23/23 - 0s - loss: 45.8305 - val_loss: 124.7607
49



<em> We see that at the end of the process "loss" has reduced to 45.8 but "val_loss" has increased until 124.8, which means that there is an overfit of Training Set, as the prediction for Test set is increasing the mean squared error.
    
    (These values will change if the fit process is executed again)
    


# Module B

## If we normalize data

This step is to normalize the data by substracting the mean and dividing by the standard deviation.

In [16]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [17]:
n_cols = predictors_norm.shape[1] # number of predictors

In [18]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)

Epoch 1/50
23/23 - 0s - loss: 1680.0623 - val_loss: 1190.3831
Epoch 2/50
23/23 - 0s - loss: 1623.2501 - val_loss: 1141.8115
Epoch 3/50
23/23 - 0s - loss: 1568.3931 - val_loss: 1095.2769
Epoch 4/50
23/23 - 0s - loss: 1516.5352 - val_loss: 1051.0221
Epoch 5/50
23/23 - 0s - loss: 1467.0304 - val_loss: 1009.3087
Epoch 6/50
23/23 - 0s - loss: 1419.0798 - val_loss: 970.9131
Epoch 7/50
23/23 - 0s - loss: 1373.8126 - val_loss: 933.5023
Epoch 8/50
23/23 - 0s - loss: 1330.2799 - val_loss: 897.7071
Epoch 9/50
23/23 - 0s - loss: 1288.5983 - val_loss: 863.9823
Epoch 10/50
23/23 - 0s - loss: 1249.3049 - val_loss: 830.8765
Epoch 11/50
23/23 - 0s - loss: 1210.5276 - val_loss: 799.9076
Epoch 12/50
23/23 - 0s - loss: 1174.0461 - val_loss: 770.2422
Epoch 13/50
23/23 - 0s - loss: 1139.5690 - val_loss: 742.1003
Epoch 14/50
23/23 - 0s - loss: 1106.1477 - val_loss: 715.7678
Epoch 15/50
23/23 - 0s - loss: 1074.6869 - val_loss: 690.4059
Epoch 16/50
23/23 - 0s - loss: 1044.1360 - val_loss: 666.5153
Epoch 17/50



Then, we see that "loss" for Training Set is finally 509.7 and "val_loss" for Test Set is 270.5. So, it seems that for Normalize data the effect of overfitting is not so evident, at least in this initial "fit".


In [19]:
for i in np.arange(50):
    model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    print (i)
    i=i+1
    

Epoch 1/50
23/23 - 0s - loss: 503.1999 - val_loss: 265.3300
Epoch 2/50
23/23 - 0s - loss: 496.8401 - val_loss: 260.0179
Epoch 3/50
23/23 - 0s - loss: 490.3987 - val_loss: 255.0013
Epoch 4/50
23/23 - 0s - loss: 484.2424 - val_loss: 250.0732
Epoch 5/50
23/23 - 0s - loss: 478.0437 - val_loss: 245.1384
Epoch 6/50
23/23 - 0s - loss: 472.0183 - val_loss: 240.5395
Epoch 7/50
23/23 - 0s - loss: 465.9464 - val_loss: 235.9006
Epoch 8/50
23/23 - 0s - loss: 460.0732 - val_loss: 231.5395
Epoch 9/50
23/23 - 0s - loss: 454.1179 - val_loss: 227.0055
Epoch 10/50
23/23 - 0s - loss: 448.1693 - val_loss: 222.6475
Epoch 11/50
23/23 - 0s - loss: 442.1348 - val_loss: 218.7320
Epoch 12/50
23/23 - 0s - loss: 436.1574 - val_loss: 214.7732
Epoch 13/50
23/23 - 0s - loss: 430.0477 - val_loss: 210.7135
Epoch 14/50
23/23 - 0s - loss: 424.0433 - val_loss: 207.0947
Epoch 15/50
23/23 - 0s - loss: 417.9027 - val_loss: 203.3722
Epoch 16/50
23/23 - 0s - loss: 411.7259 - val_loss: 200.0966
Epoch 17/50
23/23 - 0s - loss: 40

Epoch 36/50
23/23 - 0s - loss: 104.5142 - val_loss: 94.4450
Epoch 37/50
23/23 - 0s - loss: 103.9577 - val_loss: 94.2268
Epoch 38/50
23/23 - 0s - loss: 103.2913 - val_loss: 93.7310
Epoch 39/50
23/23 - 0s - loss: 102.6894 - val_loss: 93.5203
Epoch 40/50
23/23 - 0s - loss: 102.0815 - val_loss: 93.1010
Epoch 41/50
23/23 - 0s - loss: 101.4966 - val_loss: 92.7938
Epoch 42/50
23/23 - 0s - loss: 100.8666 - val_loss: 92.5975
Epoch 43/50
23/23 - 0s - loss: 100.3180 - val_loss: 92.0610
Epoch 44/50
23/23 - 0s - loss: 99.6865 - val_loss: 91.6676
Epoch 45/50
23/23 - 0s - loss: 99.1237 - val_loss: 91.5341
Epoch 46/50
23/23 - 0s - loss: 98.5448 - val_loss: 91.1636
Epoch 47/50
23/23 - 0s - loss: 97.9941 - val_loss: 90.9553
Epoch 48/50
23/23 - 0s - loss: 97.4144 - val_loss: 90.7278
Epoch 49/50
23/23 - 0s - loss: 96.8717 - val_loss: 90.0790
Epoch 50/50
23/23 - 0s - loss: 96.3128 - val_loss: 89.8118
2
Epoch 1/50
23/23 - 0s - loss: 95.8176 - val_loss: 89.6110
Epoch 2/50
23/23 - 0s - loss: 95.1691 - val_los

Epoch 26/50
23/23 - 0s - loss: 56.2255 - val_loss: 68.5842
Epoch 27/50
23/23 - 0s - loss: 56.1201 - val_loss: 68.6363
Epoch 28/50
23/23 - 0s - loss: 55.9998 - val_loss: 68.5672
Epoch 29/50
23/23 - 0s - loss: 55.7928 - val_loss: 68.7900
Epoch 30/50
23/23 - 0s - loss: 55.6470 - val_loss: 68.7293
Epoch 31/50
23/23 - 0s - loss: 55.4619 - val_loss: 68.5505
Epoch 32/50
23/23 - 0s - loss: 55.3703 - val_loss: 68.4818
Epoch 33/50
23/23 - 0s - loss: 55.1776 - val_loss: 68.9345
Epoch 34/50
23/23 - 0s - loss: 55.0432 - val_loss: 68.8239
Epoch 35/50
23/23 - 0s - loss: 54.9474 - val_loss: 68.8645
Epoch 36/50
23/23 - 0s - loss: 54.8133 - val_loss: 69.3995
Epoch 37/50
23/23 - 0s - loss: 54.6340 - val_loss: 68.9123
Epoch 38/50
23/23 - 0s - loss: 54.4994 - val_loss: 69.1864
Epoch 39/50
23/23 - 0s - loss: 54.3764 - val_loss: 68.9800
Epoch 40/50
23/23 - 0s - loss: 54.2073 - val_loss: 68.9649
Epoch 41/50
23/23 - 0s - loss: 54.0692 - val_loss: 69.2189
Epoch 42/50
23/23 - 0s - loss: 53.9087 - val_loss: 69.24

Epoch 16/50
23/23 - 0s - loss: 45.0108 - val_loss: 78.6621
Epoch 17/50
23/23 - 0s - loss: 44.9898 - val_loss: 79.2736
Epoch 18/50
23/23 - 0s - loss: 44.9451 - val_loss: 79.0626
Epoch 19/50
23/23 - 0s - loss: 44.8844 - val_loss: 79.2918
Epoch 20/50
23/23 - 0s - loss: 44.7865 - val_loss: 78.9496
Epoch 21/50
23/23 - 0s - loss: 44.7550 - val_loss: 79.3326
Epoch 22/50
23/23 - 0s - loss: 44.7164 - val_loss: 79.7813
Epoch 23/50
23/23 - 0s - loss: 44.6130 - val_loss: 79.2452
Epoch 24/50
23/23 - 0s - loss: 44.6271 - val_loss: 80.3792
Epoch 25/50
23/23 - 0s - loss: 44.5448 - val_loss: 79.6849
Epoch 26/50
23/23 - 0s - loss: 44.4616 - val_loss: 79.1052
Epoch 27/50
23/23 - 0s - loss: 44.4553 - val_loss: 79.9045
Epoch 28/50
23/23 - 0s - loss: 44.3856 - val_loss: 80.5840
Epoch 29/50
23/23 - 0s - loss: 44.3046 - val_loss: 79.8535
Epoch 30/50
23/23 - 0s - loss: 44.2617 - val_loss: 79.8666
Epoch 31/50
23/23 - 0s - loss: 44.2077 - val_loss: 80.7615
Epoch 32/50
23/23 - 0s - loss: 44.2745 - val_loss: 80.24

23/23 - 0s - loss: 37.4477 - val_loss: 99.8124
Epoch 6/50
23/23 - 0s - loss: 37.3776 - val_loss: 98.8780
Epoch 7/50
23/23 - 0s - loss: 37.2598 - val_loss: 98.9096
Epoch 8/50
23/23 - 0s - loss: 37.2459 - val_loss: 100.2275
Epoch 9/50
23/23 - 0s - loss: 37.2179 - val_loss: 99.8451
Epoch 10/50
23/23 - 0s - loss: 37.1775 - val_loss: 100.3153
Epoch 11/50
23/23 - 0s - loss: 37.0981 - val_loss: 101.5750
Epoch 12/50
23/23 - 0s - loss: 37.0561 - val_loss: 101.0589
Epoch 13/50
23/23 - 0s - loss: 37.0204 - val_loss: 101.3644
Epoch 14/50
23/23 - 0s - loss: 36.9618 - val_loss: 101.2955
Epoch 15/50
23/23 - 0s - loss: 36.9186 - val_loss: 101.2776
Epoch 16/50
23/23 - 0s - loss: 36.8790 - val_loss: 101.6014
Epoch 17/50
23/23 - 0s - loss: 36.8842 - val_loss: 100.7607
Epoch 18/50
23/23 - 0s - loss: 36.8217 - val_loss: 101.3733
Epoch 19/50
23/23 - 0s - loss: 36.7563 - val_loss: 100.1772
Epoch 20/50
23/23 - 0s - loss: 36.7145 - val_loss: 101.9332
Epoch 21/50
23/23 - 0s - loss: 36.6733 - val_loss: 102.7157


...

Epoch 26/50
23/23 - 0s - loss: 27.2419 - val_loss: 137.4937
Epoch 27/50
23/23 - 0s - loss: 27.2436 - val_loss: 137.7988
Epoch 28/50
23/23 - 0s - loss: 27.2635 - val_loss: 137.2647
Epoch 29/50
23/23 - 0s - loss: 27.2425 - val_loss: 137.3038
Epoch 30/50
23/23 - 0s - loss: 27.2691 - val_loss: 137.8601
Epoch 31/50
23/23 - 0s - loss: 27.2458 - val_loss: 137.6198
Epoch 32/50
23/23 - 0s - loss: 27.2213 - val_loss: 138.4935
Epoch 33/50
23/23 - 0s - loss: 27.2766 - val_loss: 137.9951
Epoch 34/50
23/23 - 0s - loss: 27.2909 - val_loss: 137.8398
Epoch 35/50
23/23 - 0s - loss: 27.2201 - val_loss: 137.2938
Epoch 36/50
23/23 - 0s - loss: 27.2518 - val_loss: 136.7062
Epoch 37/50
23/23 - 0s - loss: 27.2408 - val_loss: 137.9408
Epoch 38/50
23/23 - 0s - loss: 27.2341 - val_loss: 138.0516
Epoch 39/50
23/23 - 0s - loss: 27.2684 - val_loss: 138.4464
Epoch 40/50
23/23 - 0s - loss: 27.3010 - val_loss: 136.6068
Epoch 41/50
23/23 - 0s - loss: 27.2178 - val_loss: 138.9794
Epoch 42/50
23/23 - 0s - loss: 27.2922 -

Epoch 13/50
23/23 - 0s - loss: 27.0544 - val_loss: 137.5871
Epoch 14/50
23/23 - 0s - loss: 27.1247 - val_loss: 136.2378
Epoch 15/50
23/23 - 0s - loss: 27.0527 - val_loss: 136.7093
Epoch 16/50
23/23 - 0s - loss: 27.0668 - val_loss: 137.2738
Epoch 17/50
23/23 - 0s - loss: 27.1438 - val_loss: 137.4242
Epoch 18/50
23/23 - 0s - loss: 27.0626 - val_loss: 138.9761
Epoch 19/50
23/23 - 0s - loss: 27.1818 - val_loss: 137.1138
Epoch 20/50
23/23 - 0s - loss: 27.0761 - val_loss: 137.2787
Epoch 21/50
23/23 - 0s - loss: 27.0405 - val_loss: 137.9331
Epoch 22/50
23/23 - 0s - loss: 27.1084 - val_loss: 136.9223
Epoch 23/50
23/23 - 0s - loss: 27.0875 - val_loss: 137.3046
Epoch 24/50
23/23 - 0s - loss: 27.0613 - val_loss: 137.5841
Epoch 25/50
23/23 - 0s - loss: 27.0250 - val_loss: 136.7340
Epoch 26/50
23/23 - 0s - loss: 27.0344 - val_loss: 136.3502
Epoch 27/50
23/23 - 0s - loss: 27.0797 - val_loss: 137.1440
Epoch 28/50
23/23 - 0s - loss: 27.0994 - val_loss: 137.1605
Epoch 29/50
23/23 - 0s - loss: 27.0367 -

Epoch 50/50
23/23 - 0s - loss: 26.7609 - val_loss: 137.4919
43
Epoch 1/50
23/23 - 0s - loss: 26.8047 - val_loss: 136.7862
Epoch 2/50
23/23 - 0s - loss: 26.7440 - val_loss: 137.3229
Epoch 3/50
23/23 - 0s - loss: 26.7804 - val_loss: 137.6280
Epoch 4/50
23/23 - 0s - loss: 26.7670 - val_loss: 135.6586
Epoch 5/50
23/23 - 0s - loss: 26.7469 - val_loss: 137.5466
Epoch 6/50
23/23 - 0s - loss: 26.7574 - val_loss: 137.0439
Epoch 7/50
23/23 - 0s - loss: 26.7372 - val_loss: 136.5965
Epoch 8/50
23/23 - 0s - loss: 26.7518 - val_loss: 136.0836
Epoch 9/50
23/23 - 0s - loss: 26.7345 - val_loss: 135.8743
Epoch 10/50
23/23 - 0s - loss: 26.7350 - val_loss: 136.6665
Epoch 11/50
23/23 - 0s - loss: 26.7207 - val_loss: 136.1490
Epoch 12/50
23/23 - 0s - loss: 26.7137 - val_loss: 136.6438
Epoch 13/50
23/23 - 0s - loss: 26.6943 - val_loss: 136.8274
Epoch 14/50
23/23 - 0s - loss: 26.7309 - val_loss: 136.8596
Epoch 15/50
23/23 - 0s - loss: 26.7059 - val_loss: 136.3754
Epoch 16/50
23/23 - 0s - loss: 26.6724 - val_l

Epoch 37/50
23/23 - 0s - loss: 26.2640 - val_loss: 131.7354
Epoch 38/50
23/23 - 0s - loss: 26.2474 - val_loss: 131.5117
Epoch 39/50
23/23 - 0s - loss: 26.2650 - val_loss: 132.1051
Epoch 40/50
23/23 - 0s - loss: 26.2586 - val_loss: 131.2655
Epoch 41/50
23/23 - 0s - loss: 26.2311 - val_loss: 131.9951
Epoch 42/50
23/23 - 0s - loss: 26.2707 - val_loss: 131.0816
Epoch 43/50
23/23 - 0s - loss: 26.1854 - val_loss: 131.0984
Epoch 44/50
23/23 - 0s - loss: 26.2474 - val_loss: 131.7627
Epoch 45/50
23/23 - 0s - loss: 26.1999 - val_loss: 132.2710
Epoch 46/50
23/23 - 0s - loss: 26.2309 - val_loss: 131.9015
Epoch 47/50
23/23 - 0s - loss: 26.1603 - val_loss: 131.3063
Epoch 48/50
23/23 - 0s - loss: 26.1823 - val_loss: 131.8392
Epoch 49/50
23/23 - 0s - loss: 26.2088 - val_loss: 130.7764
Epoch 50/50
23/23 - 0s - loss: 26.1924 - val_loss: 130.3705
46
Epoch 1/50
23/23 - 0s - loss: 26.1906 - val_loss: 131.3324
Epoch 2/50
23/23 - 0s - loss: 26.1779 - val_loss: 131.1076
Epoch 3/50
23/23 - 0s - loss: 26.1865 -

Epoch 24/50
23/23 - 0s - loss: 25.6583 - val_loss: 130.5564
Epoch 25/50
23/23 - 0s - loss: 25.6858 - val_loss: 131.3958
Epoch 26/50
23/23 - 0s - loss: 25.6992 - val_loss: 130.3222
Epoch 27/50
23/23 - 0s - loss: 25.6184 - val_loss: 130.3569
Epoch 28/50
23/23 - 0s - loss: 25.6851 - val_loss: 130.9935
Epoch 29/50
23/23 - 0s - loss: 25.7033 - val_loss: 130.4228
Epoch 30/50
23/23 - 0s - loss: 25.6670 - val_loss: 130.8651
Epoch 31/50
23/23 - 0s - loss: 25.6383 - val_loss: 130.4538
Epoch 32/50
23/23 - 0s - loss: 25.6415 - val_loss: 130.7621
Epoch 33/50
23/23 - 0s - loss: 25.6328 - val_loss: 130.6708
Epoch 34/50
23/23 - 0s - loss: 25.6633 - val_loss: 130.9259
Epoch 35/50
23/23 - 0s - loss: 25.6780 - val_loss: 130.2531
Epoch 36/50
23/23 - 0s - loss: 25.6110 - val_loss: 130.7561
Epoch 37/50
23/23 - 0s - loss: 25.6012 - val_loss: 131.4783
Epoch 38/50
23/23 - 0s - loss: 25.6010 - val_loss: 130.9656
Epoch 39/50
23/23 - 0s - loss: 25.6046 - val_loss: 131.3428
Epoch 40/50
23/23 - 0s - loss: 25.5970 -


We see again that "loss" (mean squared error) for Training data is decreasing, and for Test data is also decreasing until 4th or 5th iteration, where tha value of "val-loss" again begins to increase.

At the end of process, mean squared error for Training set has been reduced to 25.5 (<em>less than for Module A<em>) and for Test set is increased to 130.96 !!.

(these values will change each time the process is executed)

So, again we can observe such overfit occurs.

# Module C

We will see now with epochs value of 100. Using normalized data as in module B.

We initialize the process and will apply .fit with value 100 in epochs.


In [24]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 0s - loss: 1666.2750 - val_loss: 1201.2073
Epoch 2/100
23/23 - 0s - loss: 1648.7301 - val_loss: 1190.4198
Epoch 3/100
23/23 - 0s - loss: 1630.2424 - val_loss: 1179.3394
Epoch 4/100
23/23 - 0s - loss: 1610.7958 - val_loss: 1167.4019
Epoch 5/100
23/23 - 0s - loss: 1589.8193 - val_loss: 1154.8367
Epoch 6/100
23/23 - 0s - loss: 1567.6189 - val_loss: 1141.3601
Epoch 7/100
23/23 - 0s - loss: 1543.2112 - val_loss: 1127.1639
Epoch 8/100
23/23 - 0s - loss: 1517.4357 - val_loss: 1111.5212
Epoch 9/100
23/23 - 0s - loss: 1489.5178 - val_loss: 1094.9260
Epoch 10/100
23/23 - 0s - loss: 1460.1176 - val_loss: 1077.4412
Epoch 11/100
23/23 - 0s - loss: 1428.7136 - val_loss: 1058.8126
Epoch 12/100
23/23 - 0s - loss: 1395.7584 - val_loss: 1039.8081
Epoch 13/100
23/23 - 0s - loss: 1361.5079 - val_loss: 1019.3573
Epoch 14/100
23/23 - 0s - loss: 1325.9451 - val_loss: 998.2665
Epoch 15/100
23/23 - 0s - loss: 1289.2780 - val_loss: 976.1183
Epoch 16/100
23/23 - 0s - loss: 1250.9915 - val_los

These values of "loss" and "val_loss" are lower than in Module B. So, 100 epochs could have a positive influence in training.

In [25]:
for i in np.arange(50):
    model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)
    i=i+1

Epoch 1/100
23/23 - 0s - loss: 168.0033 - val_loss: 167.5423
Epoch 2/100
23/23 - 0s - loss: 167.0257 - val_loss: 166.4655
Epoch 3/100
23/23 - 0s - loss: 166.0492 - val_loss: 165.3772
Epoch 4/100
23/23 - 0s - loss: 165.1790 - val_loss: 164.6621
Epoch 5/100
23/23 - 0s - loss: 164.1561 - val_loss: 163.2067
Epoch 6/100
23/23 - 0s - loss: 163.2856 - val_loss: 162.5350
Epoch 7/100
23/23 - 0s - loss: 162.3936 - val_loss: 161.3150
Epoch 8/100
23/23 - 0s - loss: 161.4057 - val_loss: 160.5793
Epoch 9/100
23/23 - 0s - loss: 160.5291 - val_loss: 159.5217
Epoch 10/100
23/23 - 0s - loss: 159.6733 - val_loss: 158.2364
Epoch 11/100
23/23 - 0s - loss: 158.7260 - val_loss: 157.4811
Epoch 12/100
23/23 - 0s - loss: 157.9220 - val_loss: 156.2204
Epoch 13/100
23/23 - 0s - loss: 157.1326 - val_loss: 155.5842
Epoch 14/100
23/23 - 0s - loss: 156.1199 - val_loss: 154.5714
Epoch 15/100
23/23 - 0s - loss: 155.2742 - val_loss: 152.9623
Epoch 16/100
23/23 - 0s - loss: 154.3716 - val_loss: 152.2042
Epoch 17/100
23/2

23/23 - 0s - loss: 93.7554 - val_loss: 97.4910
Epoch 35/100
23/23 - 0s - loss: 93.4112 - val_loss: 96.8161
Epoch 36/100
23/23 - 0s - loss: 93.2400 - val_loss: 96.8737
Epoch 37/100
23/23 - 0s - loss: 92.9896 - val_loss: 96.2162
Epoch 38/100
23/23 - 0s - loss: 92.8050 - val_loss: 96.3472
Epoch 39/100
23/23 - 0s - loss: 92.5745 - val_loss: 95.9958
Epoch 40/100
23/23 - 0s - loss: 92.3180 - val_loss: 95.6314
Epoch 41/100
23/23 - 0s - loss: 92.0759 - val_loss: 95.9014
Epoch 42/100
23/23 - 0s - loss: 91.7903 - val_loss: 95.4950
Epoch 43/100
23/23 - 0s - loss: 91.6122 - val_loss: 94.9348
Epoch 44/100
23/23 - 0s - loss: 91.3527 - val_loss: 95.2492
Epoch 45/100
23/23 - 0s - loss: 91.1685 - val_loss: 95.1880
Epoch 46/100
23/23 - 0s - loss: 90.9435 - val_loss: 94.8095
Epoch 47/100
23/23 - 0s - loss: 90.7330 - val_loss: 94.8227
Epoch 48/100
23/23 - 0s - loss: 90.5209 - val_loss: 94.3618
Epoch 49/100
23/23 - 0s - loss: 90.2935 - val_loss: 94.2549
Epoch 50/100
23/23 - 0s - loss: 90.0746 - val_loss: 9

Epoch 71/100
23/23 - 0s - loss: 67.3273 - val_loss: 82.7197
Epoch 72/100
23/23 - 0s - loss: 66.9430 - val_loss: 82.4950
Epoch 73/100
23/23 - 0s - loss: 66.7307 - val_loss: 82.2248
Epoch 74/100
23/23 - 0s - loss: 66.5246 - val_loss: 82.2653
Epoch 75/100
23/23 - 0s - loss: 66.3526 - val_loss: 82.6456
Epoch 76/100
23/23 - 0s - loss: 66.1670 - val_loss: 82.0694
Epoch 77/100
23/23 - 0s - loss: 65.8455 - val_loss: 82.5695
Epoch 78/100
23/23 - 0s - loss: 65.6848 - val_loss: 82.3319
Epoch 79/100
23/23 - 0s - loss: 65.4832 - val_loss: 82.0993
Epoch 80/100
23/23 - 0s - loss: 65.2773 - val_loss: 81.5875
Epoch 81/100
23/23 - 0s - loss: 65.0844 - val_loss: 82.1708
Epoch 82/100
23/23 - 0s - loss: 64.8265 - val_loss: 81.7528
Epoch 83/100
23/23 - 0s - loss: 64.5453 - val_loss: 81.7976
Epoch 84/100
23/23 - 0s - loss: 64.3178 - val_loss: 81.7189
Epoch 85/100
23/23 - 0s - loss: 64.2264 - val_loss: 81.9089
Epoch 86/100
23/23 - 0s - loss: 63.7967 - val_loss: 81.5173
Epoch 87/100
23/23 - 0s - loss: 63.5916 

Epoch 8/100
23/23 - 0s - loss: 50.6010 - val_loss: 82.5526
Epoch 9/100
23/23 - 0s - loss: 50.5712 - val_loss: 82.6559
Epoch 10/100
23/23 - 0s - loss: 50.5294 - val_loss: 82.9492
Epoch 11/100
23/23 - 0s - loss: 50.4760 - val_loss: 82.4373
Epoch 12/100
23/23 - 0s - loss: 50.4037 - val_loss: 82.7343
Epoch 13/100
23/23 - 0s - loss: 50.3510 - val_loss: 82.9373
Epoch 14/100
23/23 - 0s - loss: 50.3238 - val_loss: 82.3026
Epoch 15/100
23/23 - 0s - loss: 50.2323 - val_loss: 83.0206
Epoch 16/100
23/23 - 0s - loss: 50.1897 - val_loss: 82.7051
Epoch 17/100
23/23 - 0s - loss: 50.1112 - val_loss: 82.4400
Epoch 18/100
23/23 - 0s - loss: 50.0790 - val_loss: 82.9012
Epoch 19/100
23/23 - 0s - loss: 50.0473 - val_loss: 83.1306
Epoch 20/100
23/23 - 0s - loss: 49.9969 - val_loss: 82.3244
Epoch 21/100
23/23 - 0s - loss: 49.9090 - val_loss: 83.1230
Epoch 22/100
23/23 - 0s - loss: 49.8433 - val_loss: 82.7015
Epoch 23/100
23/23 - 0s - loss: 49.9077 - val_loss: 83.3338
Epoch 24/100
23/23 - 0s - loss: 49.7710 - 

Epoch 45/100
23/23 - 0s - loss: 44.4743 - val_loss: 86.8782
Epoch 46/100
23/23 - 0s - loss: 44.5034 - val_loss: 86.3426
Epoch 47/100
23/23 - 0s - loss: 44.5585 - val_loss: 86.2688
Epoch 48/100
23/23 - 0s - loss: 44.4723 - val_loss: 86.3612
Epoch 49/100
23/23 - 0s - loss: 44.4180 - val_loss: 86.3265
Epoch 50/100
23/23 - 0s - loss: 44.4090 - val_loss: 86.3837
Epoch 51/100
23/23 - 0s - loss: 44.3751 - val_loss: 86.7000
Epoch 52/100
23/23 - 0s - loss: 44.3138 - val_loss: 86.9129
Epoch 53/100
23/23 - 0s - loss: 44.3374 - val_loss: 87.5319
Epoch 54/100
23/23 - 0s - loss: 44.2589 - val_loss: 86.9116
Epoch 55/100
23/23 - 0s - loss: 44.2518 - val_loss: 86.5898
Epoch 56/100
23/23 - 0s - loss: 44.4148 - val_loss: 87.1658
Epoch 57/100
23/23 - 0s - loss: 44.2387 - val_loss: 87.0398
Epoch 58/100
23/23 - 0s - loss: 44.2388 - val_loss: 87.4820
Epoch 59/100
23/23 - 0s - loss: 44.1906 - val_loss: 87.9973
Epoch 60/100
23/23 - 0s - loss: 44.2229 - val_loss: 87.3707
Epoch 61/100
23/23 - 0s - loss: 44.1849 

...

Epoch 28/100
23/23 - 0s - loss: 35.5393 - val_loss: 195.5926
Epoch 29/100
23/23 - 0s - loss: 35.5257 - val_loss: 196.4395
Epoch 30/100
23/23 - 0s - loss: 35.5747 - val_loss: 196.1145
Epoch 31/100
23/23 - 0s - loss: 35.5909 - val_loss: 196.0552
Epoch 32/100
23/23 - 0s - loss: 35.5539 - val_loss: 195.5366
Epoch 33/100
23/23 - 0s - loss: 35.5110 - val_loss: 195.6677
Epoch 34/100
23/23 - 0s - loss: 35.5869 - val_loss: 195.1760
Epoch 35/100
23/23 - 0s - loss: 35.5489 - val_loss: 195.5993
Epoch 36/100
23/23 - 0s - loss: 35.6240 - val_loss: 196.8780
Epoch 37/100
23/23 - 0s - loss: 35.7968 - val_loss: 195.4632
Epoch 38/100
23/23 - 0s - loss: 35.6008 - val_loss: 195.9030
Epoch 39/100
23/23 - 0s - loss: 35.5016 - val_loss: 195.4267
Epoch 40/100
23/23 - 0s - loss: 35.5059 - val_loss: 196.2945
Epoch 41/100
23/23 - 0s - loss: 35.5169 - val_loss: 195.2091
Epoch 42/100
23/23 - 0s - loss: 35.5073 - val_loss: 196.2534
Epoch 43/100
23/23 - 0s - loss: 35.7033 - val_loss: 197.1460
Epoch 44/100
23/23 - 0s 

Epoch 63/100
23/23 - 0s - loss: 35.4555 - val_loss: 196.5315
Epoch 64/100
23/23 - 0s - loss: 35.4634 - val_loss: 197.5310
Epoch 65/100
23/23 - 0s - loss: 35.4748 - val_loss: 196.9832
Epoch 66/100
23/23 - 0s - loss: 35.4755 - val_loss: 196.9037
Epoch 67/100
23/23 - 0s - loss: 35.4548 - val_loss: 197.6081
Epoch 68/100
23/23 - 0s - loss: 35.5031 - val_loss: 198.1126
Epoch 69/100
23/23 - 0s - loss: 35.4418 - val_loss: 197.1449
Epoch 70/100
23/23 - 0s - loss: 35.5005 - val_loss: 196.9082
Epoch 71/100
23/23 - 0s - loss: 35.5299 - val_loss: 196.9733
Epoch 72/100
23/23 - 0s - loss: 35.5319 - val_loss: 198.0276
Epoch 73/100
23/23 - 0s - loss: 35.4979 - val_loss: 197.1058
Epoch 74/100
23/23 - 0s - loss: 35.4557 - val_loss: 195.8242
Epoch 75/100
23/23 - 0s - loss: 35.4601 - val_loss: 197.5597
Epoch 76/100
23/23 - 0s - loss: 35.4196 - val_loss: 196.9021
Epoch 77/100
23/23 - 0s - loss: 35.5330 - val_loss: 197.1083
Epoch 78/100
23/23 - 0s - loss: 35.4415 - val_loss: 197.2304
Epoch 79/100
23/23 - 0s 

Epoch 98/100
23/23 - 0s - loss: 35.3716 - val_loss: 196.9286
Epoch 99/100
23/23 - 0s - loss: 35.3906 - val_loss: 196.6525
Epoch 100/100
23/23 - 0s - loss: 35.3607 - val_loss: 196.7478
Epoch 1/100
23/23 - 0s - loss: 35.3872 - val_loss: 197.0261
Epoch 2/100
23/23 - 0s - loss: 35.4104 - val_loss: 197.5435
Epoch 3/100
23/23 - 0s - loss: 35.3676 - val_loss: 196.7602
Epoch 4/100
23/23 - 0s - loss: 35.3771 - val_loss: 196.5578
Epoch 5/100
23/23 - 0s - loss: 35.4219 - val_loss: 196.8490
Epoch 6/100
23/23 - 0s - loss: 35.3609 - val_loss: 196.1493
Epoch 7/100
23/23 - 0s - loss: 35.4776 - val_loss: 196.3851
Epoch 8/100
23/23 - 0s - loss: 35.3506 - val_loss: 196.7030
Epoch 9/100
23/23 - 0s - loss: 35.4351 - val_loss: 196.6160
Epoch 10/100
23/23 - 0s - loss: 35.3712 - val_loss: 197.0303
Epoch 11/100
23/23 - 0s - loss: 35.3844 - val_loss: 195.6997
Epoch 12/100
23/23 - 0s - loss: 35.3915 - val_loss: 196.2335
Epoch 13/100
23/23 - 0s - loss: 35.3872 - val_loss: 196.2523
Epoch 14/100
23/23 - 0s - loss: 

Epoch 33/100
23/23 - 0s - loss: 35.3513 - val_loss: 195.9618
Epoch 34/100
23/23 - 0s - loss: 35.3854 - val_loss: 197.4909
Epoch 35/100
23/23 - 0s - loss: 35.4256 - val_loss: 195.5039
Epoch 36/100
23/23 - 0s - loss: 35.3148 - val_loss: 197.1770
Epoch 37/100
23/23 - 0s - loss: 35.3966 - val_loss: 197.1551
Epoch 38/100
23/23 - 0s - loss: 35.3427 - val_loss: 195.9037
Epoch 39/100
23/23 - 0s - loss: 35.3325 - val_loss: 196.6605
Epoch 40/100
23/23 - 0s - loss: 35.3516 - val_loss: 196.8918
Epoch 41/100
23/23 - 0s - loss: 35.3938 - val_loss: 195.9071
Epoch 42/100
23/23 - 0s - loss: 35.3607 - val_loss: 197.4260
Epoch 43/100
23/23 - 0s - loss: 35.4008 - val_loss: 196.9221
Epoch 44/100
23/23 - 0s - loss: 35.3771 - val_loss: 196.4101
Epoch 45/100
23/23 - 0s - loss: 35.3762 - val_loss: 195.8067
Epoch 46/100
23/23 - 0s - loss: 35.5103 - val_loss: 197.7482
Epoch 47/100
23/23 - 0s - loss: 35.4108 - val_loss: 195.2678
Epoch 48/100
23/23 - 0s - loss: 35.3577 - val_loss: 196.2590
Epoch 49/100
23/23 - 0s 

Epoch 68/100
23/23 - 0s - loss: 35.3395 - val_loss: 196.5969
Epoch 69/100
23/23 - 0s - loss: 35.2935 - val_loss: 197.2847
Epoch 70/100
23/23 - 0s - loss: 35.2429 - val_loss: 196.8060
Epoch 71/100
23/23 - 0s - loss: 35.2435 - val_loss: 196.4200
Epoch 72/100
23/23 - 0s - loss: 35.2878 - val_loss: 197.4968
Epoch 73/100
23/23 - 0s - loss: 35.3117 - val_loss: 195.5270
Epoch 74/100
23/23 - 0s - loss: 35.2512 - val_loss: 197.4008
Epoch 75/100
23/23 - 0s - loss: 35.2514 - val_loss: 197.7839
Epoch 76/100
23/23 - 0s - loss: 35.2606 - val_loss: 197.8747
Epoch 77/100
23/23 - 0s - loss: 35.2700 - val_loss: 196.8852
Epoch 78/100
23/23 - 0s - loss: 35.2654 - val_loss: 196.1562
Epoch 79/100
23/23 - 0s - loss: 35.3037 - val_loss: 196.2915
Epoch 80/100
23/23 - 0s - loss: 35.2467 - val_loss: 195.3163
Epoch 81/100
23/23 - 0s - loss: 35.2717 - val_loss: 196.7833
Epoch 82/100
23/23 - 0s - loss: 35.2624 - val_loss: 195.7029
Epoch 83/100
23/23 - 0s - loss: 35.2917 - val_loss: 196.1188
Epoch 84/100
23/23 - 0s 


With this value of 100 epochs the result for loss (squared mean error) for Training set has been in this case a little higher than for 50 (mean_squared_error around 35 instead 25), and Test set is again having the same evolution after the 4th or 5th iteration, so there is again some overfit as at the end mean_squared_error (val_loss) of Test set (validation) increases until 197.3 (in this sample)



# Module D



### Finally we'll the define again the model for 3 hidden layers with 10 nodes


In [26]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [27]:
# build the model
model = regression_model()

In [28]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)

Epoch 1/50
23/23 - 0s - loss: 1703.1598 - val_loss: 1230.7943
Epoch 2/50
23/23 - 0s - loss: 1681.6068 - val_loss: 1215.1412
Epoch 3/50
23/23 - 0s - loss: 1654.1990 - val_loss: 1193.2472
Epoch 4/50
23/23 - 0s - loss: 1610.4514 - val_loss: 1159.2886
Epoch 5/50
23/23 - 0s - loss: 1540.6837 - val_loss: 1104.9523
Epoch 6/50
23/23 - 0s - loss: 1429.9546 - val_loss: 1026.3693
Epoch 7/50
23/23 - 0s - loss: 1272.9902 - val_loss: 917.1235
Epoch 8/50
23/23 - 0s - loss: 1064.8857 - val_loss: 780.3760
Epoch 9/50
23/23 - 0s - loss: 819.9774 - val_loss: 625.3066
Epoch 10/50
23/23 - 0s - loss: 582.9695 - val_loss: 471.4433
Epoch 11/50
23/23 - 0s - loss: 419.5711 - val_loss: 352.2292
Epoch 12/50
23/23 - 0s - loss: 321.2989 - val_loss: 289.1343
Epoch 13/50
23/23 - 0s - loss: 276.9084 - val_loss: 245.3046
Epoch 14/50
23/23 - 0s - loss: 248.2982 - val_loss: 221.1378
Epoch 15/50
23/23 - 0s - loss: 229.7069 - val_loss: 205.9811
Epoch 16/50
23/23 - 0s - loss: 218.1201 - val_loss: 194.6468
Epoch 17/50
23/23 -

Thease are the best values (lowest) for mean_sequared_error in Training and Test Set in first iteration of .fit. So having 3 hidden layers can be positive for the process. Let's see repeating again 50 times

In [29]:
for i in np.arange(49):
    model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)
    print(i)
    i=i+1

Epoch 1/50
23/23 - 0s - loss: 109.7156 - val_loss: 127.9380
Epoch 2/50
23/23 - 0s - loss: 109.2253 - val_loss: 128.5667
Epoch 3/50
23/23 - 0s - loss: 108.0563 - val_loss: 127.9674
Epoch 4/50
23/23 - 0s - loss: 107.7534 - val_loss: 127.4857
Epoch 5/50
23/23 - 0s - loss: 106.9638 - val_loss: 124.6713
Epoch 6/50
23/23 - 0s - loss: 106.2799 - val_loss: 127.6238
Epoch 7/50
23/23 - 0s - loss: 105.3615 - val_loss: 125.2796
Epoch 8/50
23/23 - 0s - loss: 104.6699 - val_loss: 123.7656
Epoch 9/50
23/23 - 0s - loss: 104.2646 - val_loss: 126.3728
Epoch 10/50
23/23 - 0s - loss: 103.6677 - val_loss: 124.0943
Epoch 11/50
23/23 - 0s - loss: 102.9480 - val_loss: 125.0119
Epoch 12/50
23/23 - 0s - loss: 102.7224 - val_loss: 122.6204
Epoch 13/50
23/23 - 0s - loss: 101.7801 - val_loss: 124.6829
Epoch 14/50
23/23 - 0s - loss: 101.4054 - val_loss: 123.3464
Epoch 15/50
23/23 - 0s - loss: 100.8504 - val_loss: 121.8520
Epoch 16/50
23/23 - 0s - loss: 100.1985 - val_loss: 124.0132
Epoch 17/50
23/23 - 0s - loss: 99

Epoch 38/50
23/23 - 0s - loss: 77.7950 - val_loss: 102.6585
Epoch 39/50
23/23 - 0s - loss: 78.1243 - val_loss: 97.7868
Epoch 40/50
23/23 - 0s - loss: 77.7604 - val_loss: 101.2250
Epoch 41/50
23/23 - 0s - loss: 77.7068 - val_loss: 97.7674
Epoch 42/50
23/23 - 0s - loss: 77.4492 - val_loss: 101.0375
Epoch 43/50
23/23 - 0s - loss: 77.6395 - val_loss: 99.3727
Epoch 44/50
23/23 - 0s - loss: 79.1149 - val_loss: 102.6777
Epoch 45/50
23/23 - 0s - loss: 77.3970 - val_loss: 100.6936
Epoch 46/50
23/23 - 0s - loss: 77.3146 - val_loss: 102.9137
Epoch 47/50
23/23 - 0s - loss: 77.5907 - val_loss: 103.5377
Epoch 48/50
23/23 - 0s - loss: 77.1557 - val_loss: 100.0723
Epoch 49/50
23/23 - 0s - loss: 77.1150 - val_loss: 99.0343
Epoch 50/50
23/23 - 0s - loss: 77.3286 - val_loss: 101.1538
2
Epoch 1/50
23/23 - 0s - loss: 77.2133 - val_loss: 99.1659
Epoch 2/50
23/23 - 0s - loss: 76.6667 - val_loss: 99.7083
Epoch 3/50
23/23 - 0s - loss: 76.8647 - val_loss: 98.9396
Epoch 4/50
23/23 - 0s - loss: 76.6425 - val_loss

Epoch 27/50
23/23 - 0s - loss: 50.2384 - val_loss: 90.4587
Epoch 28/50
23/23 - 0s - loss: 50.3402 - val_loss: 93.0937
Epoch 29/50
23/23 - 0s - loss: 49.7225 - val_loss: 94.9323
Epoch 30/50
23/23 - 0s - loss: 49.5669 - val_loss: 97.6490
Epoch 31/50
23/23 - 0s - loss: 49.0335 - val_loss: 94.5967
Epoch 32/50
23/23 - 0s - loss: 49.1317 - val_loss: 95.0993
Epoch 33/50
23/23 - 0s - loss: 49.1491 - val_loss: 89.8502
Epoch 34/50
23/23 - 0s - loss: 48.6766 - val_loss: 94.0572
Epoch 35/50
23/23 - 0s - loss: 48.2927 - val_loss: 92.7020
Epoch 36/50
23/23 - 0s - loss: 48.1492 - val_loss: 94.6704
Epoch 37/50
23/23 - 0s - loss: 47.6536 - val_loss: 98.0271
Epoch 38/50
23/23 - 0s - loss: 47.4723 - val_loss: 95.9854
Epoch 39/50
23/23 - 0s - loss: 47.3324 - val_loss: 86.3122
Epoch 40/50
23/23 - 0s - loss: 47.4898 - val_loss: 95.0138
Epoch 41/50
23/23 - 0s - loss: 46.9111 - val_loss: 97.3463
Epoch 42/50
23/23 - 0s - loss: 46.8495 - val_loss: 97.2595
Epoch 43/50
23/23 - 0s - loss: 46.0672 - val_loss: 98.56

Epoch 15/50
23/23 - 0s - loss: 29.4740 - val_loss: 142.6500
Epoch 16/50
23/23 - 0s - loss: 29.3313 - val_loss: 131.0811
Epoch 17/50
23/23 - 0s - loss: 29.1824 - val_loss: 140.6591
Epoch 18/50
23/23 - 0s - loss: 29.2548 - val_loss: 133.8290
Epoch 19/50
23/23 - 0s - loss: 29.2835 - val_loss: 142.7950
Epoch 20/50
23/23 - 0s - loss: 29.3814 - val_loss: 134.0086
Epoch 21/50
23/23 - 0s - loss: 29.2060 - val_loss: 149.3751
Epoch 22/50
23/23 - 0s - loss: 29.1885 - val_loss: 119.5240
Epoch 23/50
23/23 - 0s - loss: 29.4168 - val_loss: 143.8769
Epoch 24/50
23/23 - 0s - loss: 29.4346 - val_loss: 132.6726
Epoch 25/50
23/23 - 0s - loss: 29.1797 - val_loss: 139.5471
Epoch 26/50
23/23 - 0s - loss: 29.1626 - val_loss: 135.2576
Epoch 27/50
23/23 - 0s - loss: 29.1499 - val_loss: 137.9166
Epoch 28/50
23/23 - 0s - loss: 29.2852 - val_loss: 138.1268
Epoch 29/50
23/23 - 0s - loss: 29.1106 - val_loss: 134.9316
Epoch 30/50
23/23 - 0s - loss: 29.0177 - val_loss: 138.2833
Epoch 31/50
23/23 - 0s - loss: 28.9518 -

Epoch 2/50
23/23 - 0s - loss: 26.4573 - val_loss: 116.7281
Epoch 3/50
23/23 - 0s - loss: 27.4941 - val_loss: 127.8013
Epoch 4/50
23/23 - 0s - loss: 26.4351 - val_loss: 129.9410
Epoch 5/50
23/23 - 0s - loss: 26.2477 - val_loss: 131.7436
Epoch 6/50
23/23 - 0s - loss: 26.3460 - val_loss: 126.4052
Epoch 7/50
23/23 - 0s - loss: 26.4028 - val_loss: 125.9539
Epoch 8/50
23/23 - 0s - loss: 26.3152 - val_loss: 129.6728
Epoch 9/50
23/23 - 0s - loss: 26.4984 - val_loss: 126.1087
Epoch 10/50
23/23 - 0s - loss: 26.2606 - val_loss: 119.1219
Epoch 11/50
23/23 - 0s - loss: 26.4012 - val_loss: 124.0520
Epoch 12/50
23/23 - 0s - loss: 26.0948 - val_loss: 129.6609
Epoch 13/50
23/23 - 0s - loss: 26.3410 - val_loss: 128.2856
Epoch 14/50
23/23 - 0s - loss: 26.3546 - val_loss: 120.1221
Epoch 15/50
23/23 - 0s - loss: 26.3988 - val_loss: 124.7235
Epoch 16/50
23/23 - 0s - loss: 26.4949 - val_loss: 117.6568
Epoch 17/50
23/23 - 0s - loss: 26.1644 - val_loss: 125.1323
Epoch 18/50
23/23 - 0s - loss: 26.2444 - val_los

...

Epoch 37/50
23/23 - 0s - loss: 17.9023 - val_loss: 121.7678
Epoch 38/50
23/23 - 0s - loss: 17.2012 - val_loss: 125.1001
Epoch 39/50
23/23 - 0s - loss: 17.0814 - val_loss: 124.3158
Epoch 40/50
23/23 - 0s - loss: 17.3107 - val_loss: 122.8804
Epoch 41/50
23/23 - 0s - loss: 17.6624 - val_loss: 129.5984
Epoch 42/50
23/23 - 0s - loss: 16.9241 - val_loss: 133.0802
Epoch 43/50
23/23 - 0s - loss: 17.4128 - val_loss: 120.4029
Epoch 44/50
23/23 - 0s - loss: 17.1160 - val_loss: 122.0711
Epoch 45/50
23/23 - 0s - loss: 17.0944 - val_loss: 124.6110
Epoch 46/50
23/23 - 0s - loss: 17.7062 - val_loss: 124.8920
Epoch 47/50
23/23 - 0s - loss: 17.3623 - val_loss: 127.1048
Epoch 48/50
23/23 - 0s - loss: 17.1533 - val_loss: 126.5279
Epoch 49/50
23/23 - 0s - loss: 17.1105 - val_loss: 124.1363
Epoch 50/50
23/23 - 0s - loss: 16.9735 - val_loss: 129.4260
35
Epoch 1/50
23/23 - 0s - loss: 17.3284 - val_loss: 122.1816
Epoch 2/50
23/23 - 0s - loss: 17.0285 - val_loss: 123.6197
Epoch 3/50
23/23 - 0s - loss: 17.0273 -

Epoch 24/50
23/23 - 0s - loss: 16.5472 - val_loss: 139.8169
Epoch 25/50
23/23 - 0s - loss: 16.0839 - val_loss: 134.5637
Epoch 26/50
23/23 - 0s - loss: 16.3493 - val_loss: 139.7027
Epoch 27/50
23/23 - 0s - loss: 16.7521 - val_loss: 135.9011
Epoch 28/50
23/23 - 0s - loss: 16.3274 - val_loss: 135.9000
Epoch 29/50
23/23 - 0s - loss: 16.2061 - val_loss: 134.8555
Epoch 30/50
23/23 - 0s - loss: 16.5972 - val_loss: 130.5032
Epoch 31/50
23/23 - 0s - loss: 16.5519 - val_loss: 137.2848
Epoch 32/50
23/23 - 0s - loss: 16.3087 - val_loss: 134.8724
Epoch 33/50
23/23 - 0s - loss: 16.3581 - val_loss: 135.3567
Epoch 34/50
23/23 - 0s - loss: 16.2146 - val_loss: 138.1952
Epoch 35/50
23/23 - 0s - loss: 16.1794 - val_loss: 140.1415
Epoch 36/50
23/23 - 0s - loss: 16.4912 - val_loss: 140.0634
Epoch 37/50
23/23 - 0s - loss: 16.4309 - val_loss: 134.4693
Epoch 38/50
23/23 - 0s - loss: 16.3321 - val_loss: 138.8194
Epoch 39/50
23/23 - 0s - loss: 16.2293 - val_loss: 140.5669
Epoch 40/50
23/23 - 0s - loss: 16.2644 -

Epoch 11/50
23/23 - 0s - loss: 15.5500 - val_loss: 152.7823
Epoch 12/50
23/23 - 0s - loss: 15.9489 - val_loss: 158.9556
Epoch 13/50
23/23 - 0s - loss: 15.9094 - val_loss: 154.1291
Epoch 14/50
23/23 - 0s - loss: 15.7057 - val_loss: 150.6741
Epoch 15/50
23/23 - 0s - loss: 15.5746 - val_loss: 156.3261
Epoch 16/50
23/23 - 0s - loss: 15.7773 - val_loss: 150.7741
Epoch 17/50
23/23 - 0s - loss: 15.8082 - val_loss: 154.4688
Epoch 18/50
23/23 - 0s - loss: 16.0103 - val_loss: 153.9513
Epoch 19/50
23/23 - 0s - loss: 15.6018 - val_loss: 147.1588
Epoch 20/50
23/23 - 0s - loss: 15.8372 - val_loss: 153.0347
Epoch 21/50
23/23 - 0s - loss: 15.9550 - val_loss: 152.9678
Epoch 22/50
23/23 - 0s - loss: 15.6889 - val_loss: 150.0606
Epoch 23/50
23/23 - 0s - loss: 15.4476 - val_loss: 152.4230
Epoch 24/50
23/23 - 0s - loss: 15.7474 - val_loss: 159.8988
Epoch 25/50
23/23 - 0s - loss: 15.7179 - val_loss: 150.1717
Epoch 26/50
23/23 - 0s - loss: 16.0838 - val_loss: 154.2311
Epoch 27/50
23/23 - 0s - loss: 15.4355 -

Epoch 48/50
23/23 - 0s - loss: 14.7619 - val_loss: 151.0468
Epoch 49/50
23/23 - 0s - loss: 14.9917 - val_loss: 156.4317
Epoch 50/50
23/23 - 0s - loss: 14.9564 - val_loss: 154.8994
43
Epoch 1/50
23/23 - 0s - loss: 14.9189 - val_loss: 154.7976
Epoch 2/50
23/23 - 0s - loss: 15.0739 - val_loss: 156.5867
Epoch 3/50
23/23 - 0s - loss: 15.2274 - val_loss: 157.3169
Epoch 4/50
23/23 - 0s - loss: 14.9701 - val_loss: 151.3024
Epoch 5/50
23/23 - 0s - loss: 15.0113 - val_loss: 156.3688
Epoch 6/50
23/23 - 0s - loss: 14.9653 - val_loss: 153.0200
Epoch 7/50
23/23 - 0s - loss: 15.1514 - val_loss: 156.0852
Epoch 8/50
23/23 - 0s - loss: 15.1854 - val_loss: 156.1163
Epoch 9/50
23/23 - 0s - loss: 15.0092 - val_loss: 154.1171
Epoch 10/50
23/23 - 0s - loss: 15.3240 - val_loss: 153.9167
Epoch 11/50
23/23 - 0s - loss: 14.9427 - val_loss: 161.5148
Epoch 12/50
23/23 - 0s - loss: 15.3637 - val_loss: 160.7764
Epoch 13/50
23/23 - 0s - loss: 14.9180 - val_loss: 150.8167
Epoch 14/50
23/23 - 0s - loss: 14.9723 - val_l

Epoch 35/50
23/23 - 0s - loss: 14.5784 - val_loss: 162.6150
Epoch 36/50
23/23 - 0s - loss: 15.3289 - val_loss: 162.1187
Epoch 37/50
23/23 - 0s - loss: 14.7740 - val_loss: 162.4032
Epoch 38/50
23/23 - 0s - loss: 14.6617 - val_loss: 155.7165
Epoch 39/50
23/23 - 0s - loss: 14.4871 - val_loss: 158.7749
Epoch 40/50
23/23 - 0s - loss: 14.5566 - val_loss: 164.7248
Epoch 41/50
23/23 - 0s - loss: 14.8874 - val_loss: 157.1920
Epoch 42/50
23/23 - 0s - loss: 14.4260 - val_loss: 161.4760
Epoch 43/50
23/23 - 0s - loss: 14.6553 - val_loss: 155.1213
Epoch 44/50
23/23 - 0s - loss: 14.3965 - val_loss: 152.3110
Epoch 45/50
23/23 - 0s - loss: 14.8374 - val_loss: 163.0791
Epoch 46/50
23/23 - 0s - loss: 14.6782 - val_loss: 154.2400
Epoch 47/50
23/23 - 0s - loss: 14.6418 - val_loss: 156.4708
Epoch 48/50
23/23 - 0s - loss: 14.4779 - val_loss: 158.2778
Epoch 49/50
23/23 - 0s - loss: 14.2974 - val_loss: 145.0541
Epoch 50/50
23/23 - 0s - loss: 15.5596 - val_loss: 153.4762
46
Epoch 1/50
23/23 - 0s - loss: 15.7634

Again we see that Training set is reducing mean squared error, obtaining the lowest value mean squared error (14.5),

but Test set experiences a similar evolution increasing values of val_loss (169.6) at the end of this process.